# HOMEWORK 6: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming)

We will focus only on the Object Classification task for this homework.

In this homework, you are asked compare different text classification models in terms of accuracy and inference time.

You will need to build 3 different models.

1. A model based on tf-idf
2. A model based on MUSE
3. A model based on wangchanBERTa

**You will be ask to submit 3 different files (.pdf from .ipynb) that does the 3 different models. Finally, answer the accuracy and runtime numbers in MCV.**

This homework is quite free form, and your answer may vary. We hope that the processing during the course of this assignment will make you think more about the design choices in text classification.

In [18]:
# !wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

In [19]:
# !pip install pythainlp

## Import Libs

In [20]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score
import time

## Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [21]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

Let's preview the data.

In [22]:
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 0.1:
You will have to remove unwanted label duplications as well as duplications in text inputs.
Also, you will have to trim out unwanted whitespaces from the text inputs.
This shouldn't be too hard, as you have already seen it in the demo.



In [23]:
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [24]:
# TODO.1: Data cleaning
clean_time = 0
t_start = time.time()
data_df['clean Sentence Utterance'] = data_df['Sentence Utterance'].str.strip().copy()
# data_df['clean Action'] = data_df['Action'].str.lower().copy()
data_df['clean Object'] = data_df['Object'].str.lower().copy()

# data_df.drop_duplicates("Sentence Utterance", keep="first", inplace=True)
data_df.drop_duplicates("clean Sentence Utterance", keep="first", inplace=True)

data_df.drop('Sentence Utterance', axis=1, inplace=True)
data_df.drop('Action', axis=1, inplace=True)
data_df.drop('Object', axis=1, inplace=True)
t_end = time.time()
clean_time += t_end - t_start


data_df.describe()

# idx = 1
# print(f'"{data_df["Sentence Utterance"][idx]}"')
# print(f'"{data_df["clean Sentence Utterance"][idx]}"')


,clean Sentence Utterance,clean Object
count,13367,13367
unique,13367,26
top,สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ,service
freq,1,2108


In [25]:
t_start = time.time()
data = data_df.to_numpy()
unique_label = data_df['clean Object'].unique()

label_2_num = dict(zip(unique_label, range(len(unique_label))))
num_2_label = dict(zip(range(len(unique_label)), unique_label))

# display(label_2_num)
# display(num_2_label)

# display(data[:, 1])
data[:, 1] = np.vectorize(label_2_num.get)(data[:, 1])
# display(data[:, 1])

t_end = time.time()
clean_time += t_end - t_start

Split data into train, valdation, and test sets (normally the ratio will be 80:10:10 , respectively). We recommend to use train_test_spilt from scikit-learn to split the data into train, validation, test set.

In addition, it should split the data that distribution of the labels in train, validation, test set are similar. There is **stratify** option to handle this issue.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Make sure the same data splitting is used for all models.

In [26]:
bin_label = np.bincount(np.array(data[:, 1], dtype=int))
# print(data[:, 1])
print(bin_label)

[ 641 1791  730 1786  581 2108  246 1478  327  540  173 1142  280   22
  246  248  296  231   50  206   49   79   36   67    4   10]


In [27]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
t_start = time.time()
sss_train_valtest = StratifiedShuffleSplit(n_splits=1, test_size=1/10, random_state=42)
sss_val_test = StratifiedShuffleSplit(n_splits=1, test_size=1/9, random_state=42)

# print(data.shape)
trainval_idx, test_idx = next(sss_train_valtest.split(data[:, 0], data[:, 1]))
trainval_raw = data[trainval_idx]
test_raw = data[test_idx]
# print(trainval_raw.shape, test_raw.shape)

train_idx, val_idx = next(sss_val_test.split(trainval_raw[:, 0], trainval_raw[:, 1]))
train_raw = trainval_raw[train_idx]
val_raw = trainval_raw[val_idx]

# print(train_raw.shape, val_raw.shape, test_raw.shape)
t_end = time.time()
clean_time += t_end - t_start

In [28]:
import pythainlp
from pythainlp import word_tokenize
token_time = 0
t_start = time.time()

thai_stopwords = pythainlp.corpus.thai_stopwords()
thai_stopwords = list(thai_stopwords)
tokenizer = pythainlp.tokenize.word_tokenize

t_end = time.time()
token_time += t_end - t_start
print(thai_stopwords)

print(tokenizer('ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้'))

['ครั้งครา', 'ที่นั้น', 'แค่ไหน', 'ช้าๆ', 'ทั้งนี้', 'กลุ่มก้อน', 'เหล่านั้น', 'จับ', 'แห่งไหน', 'เมื่อนั้น', 'พร้อมที่', 'เดียวกัน', 'ข้าฯ', 'อาจเป็น', 'เช่นดังก่อน', 'จนแม้', 'นู่น', 'พวกมัน', 'เมื่อคืน', 'ทุกคน', 'อันได้แก่', 'ครั้ง', 'เป็นที่', 'ตลอดปี', 'พบ', 'เฉพาะ', 'พวกคุณ', 'เท่าที่', 'วันนั้น', 'จนบัดนี้', 'กัน', 'หรือ', 'เช่นเดียวกับ', 'ไกล', 'นะ', 'กว้างๆ', 'จึง', 'พึ่ง', 'เฉยๆ', 'แต่ก่อน', 'ก็แค่', 'ฝ่ายใด', 'มั้ยเนี่ย', 'เป็นแต่', 'มุ่งหมาย', 'บ่อยๆ', 'เช่นเมื่อ', 'ยาว', 'และ', 'คล้ายกัน', 'ใหญ่ๆ', 'ถูก', 'นักๆ', 'ไม่ค่อยเป็น', 'สิ่งไหน', 'ทุกที', 'ครั้งหลัง', 'ครั้งหลังสุด', 'เริ่ม', 'เช่นที่เคย', 'ใครๆ', 'ทุกครั้ง', 'ก่อนหน้านี้', 'แต่ไหน', 'ความ', 'พอ', 'ครบครัน', 'นิดๆ', 'อัน', 'นี่', 'นั้น', 'ฉัน', 'สั้น', 'สมัย', 'จวบกับ', 'ครั้งนี้', 'ร่วมด้วย', 'ข้างเคียง', 'เพียงเพื่อ', 'ที่', 'ขาด', 'สุดๆ', 'ใกล้', 'ทำๆ', 'เลย', 'ใช่ไหม', 'รือว่า', 'กำลังจะ', 'มิใช่', 'นอกจากนั้น', 'เต็มไปหมด', 'แยะ', 'เป็นแต่เพียง', 'พอแล้ว', 'ไม่ว่า', 'พอควร', 'สำคัญ', 'เสียแล้ว', 'รือ', 'กับ'

# Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans: I use pythainlp because dataset is thai language, this tokenizer might suit tokenize task for thai sentence.**

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans: From the experiments, Used stop words yield acuuracy on test set at 0.69, but without stop words reach 0.74. Each sentence in dataset might too short so stop words cut out many context of each sentence.**

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans: Occuring 8193 times and there're uniquely 473 words**

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(tokenizer=tokenizer, stop_words=None)
t_start = time.time()
vectorizer = TfidfVectorizer(tokenizer=tokenizer, stop_words=thai_stopwords)

X_train = vectorizer.fit_transform(train_raw[:, 0])
X_val = vectorizer.transform(val_raw[:, 0])
X_test = vectorizer.transform(test_raw[:, 0])

y_train = train_raw[:, 1].astype(int)
y_val = val_raw[:, 1].astype(int)
y_test = test_raw[:, 1].astype(int)

t_end = time.time()
token_time += t_end - t_start

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

/home/andre/anaconda3/envs/ML10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/andre/anaconda3/envs/ML10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['กระไร', 'กาลนาน', 'ชิ้น', 'ดังที่', 'ดี', 'ดีกว่า', 'ด้อย', 'ตัว', 'ต่อไป', 'ถัดไป', 'ทั่วถึง', 'ทำ', 'ที่จะ', 'ท่าน', 'ท้าย', 'นา', 'บอ', 'บัด', 'ระยะเวลา', 'ล่ะ', 'วันวาน', 'สม', 'สมบูรณ์', 'สํา', 'หน้า', 'หรับ', 'หา', 'อย', 'เกี่ยว', 'เก่า', 'เดี๋ยวนี้', 'เย็น', 'เล่า', 'เสมือน', 'เหมือนกัน', 'แด่', 'แม้น', 'แหล่', 'โง้น', 'โน้น', 'ใด', 'ไว', 'ไหม', '\ufeff'] not in stop_words.
  warnings.warn(


(10693, 3286) (1337, 3286) (1337, 3286)
(10693,) (1337,) (1337,)


In [30]:
example = train_raw[4, 0]
print(f"'{example}'")
print(vectorizer.transform([example]))
print(vectorizer.get_feature_names_out()[np.where(vectorizer.transform([example]).toarray()[0] > 0)])

'คะจะรบกวนสอบถามนิดนึงว่า พอดีใช้โทรศัพท์บีบีใช้ไหม แล้วเล่น ไลน์ ได้ไหม'
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 3286)>
  Coords	Values
  (0, 1)	0.3001256331419422
  (0, 1480)	0.267998540180016
  (0, 1555)	0.5518381273514346
  (0, 1880)	0.23692016311205774
  (0, 2191)	0.12867156319941367
  (0, 2858)	0.2278119799118207
  (0, 3136)	0.20822971804046236
  (0, 3258)	0.3808233638797442
  (0, 3265)	0.4696851977498628
[' ' 'นิดนึง' 'บี' 'รบกวน' 'สอบถาม' 'เล่น' 'โทรศัพท์' 'ไลน์' 'ไหม']


In [31]:
from sklearn.linear_model import LogisticRegression
train_time = 0
t_start = time.time()

clf = LogisticRegression(max_iter=5000)
clf.fit(X_train, y_train)

t_end = time.time()
train_time += t_end - t_start

In [32]:
print("Accuracy on test set:", clf.score(X_test, y_test))

infer_time = 0
t_start = time.time()
y_pred = clf.predict(X_test)
t_end = time.time()
infer_time += t_end - t_start

Accuracy on test set: 0.6925953627524308


In [33]:
oov_list = [] 
for sen in [tokenizer(sent) for sent in test_raw[:, 0]]:
    for word in sen:
        if word not in vectorizer.get_feature_names_out():
            # print(f'"{word}" not in vocab')
            oov_list.append(word)
print(len(oov_list))
oov_list_np = np.array(oov_list)
print(np.unique(oov_list_np).shape)

In [17]:
print(f"Cleaning time: {clean_time:.5f} s")
print(f"Tokenization time: {token_time:.5f} s")
print(f"Training time: {train_time:.5f} s")
print(f"Inference time: {infer_time:.5f} s")

Cleaning time: 0.01942 s
Tokenization time: 0.61159 s
Training time: 8.57086 s
Inference time: 0.00037 s


# Comparison

After you have completed the 3 models, compare the accuracy, ease of implementation, and inference speed (from cleaning, tokenization, till model compute) between the three models in mycourseville.